<a href="https://colab.research.google.com/github/ParitKansal/GenAI/blob/main/LangChain_2(MultiDataframeAgents).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***In LangChain, the Agent can access many dataframes at once***

## **01: Install All the Required Packages**

In [1]:
!pip install langchain langchain_experimental
!pip install watermark
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.6 MB/s eta 0:00:00


## **02: Import All The Required Libraries**

In [7]:
import os
import warnings
warnings.filterwarnings("ignore")

In [3]:
pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.2 MB/s eta 0:00:00


In [4]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import OpenAI

In [5]:
import pandas as pd

## **03: Setup the Environment**


In [8]:
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

## **04: Data**

In [9]:
url = "https://raw.githubusercontent.com/adamerose/datasets/master/titanic.csv"
df=pd.read_csv(url)
print(df.shape)
df.head()

(891, 15)


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## **05 : Single DataFrame Example**

### **Agent can interact with Single DataFrame**

In [10]:
llm = OpenAI(temperature=0)

In [14]:

agent = create_pandas_dataframe_agent(llm, df, verbose=True, allow_dangerous_code = True)

In [16]:
agent.invoke("How many rows are there")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows in the dataframe
Action: python_repl_ast
Action Input: len(df)891891 is the number of rows in the dataframe
Final Answer: 891

> Finished chain.


{'input': 'How many rows are there', 'output': '891'}

In [17]:
agent.invoke("How many people have more than 23 age")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include rows where the age is greater than 23, and then count the number of rows.
Action: python_repl_ast
Action Input: df[df['age'] > 23].count()survived       468
pclass         468
sex            468
age            468
sibsp          468
parch          468
fare           468
embarked       466
class          468
who            468
adult_male     468
deck           145
embark_town    466
alive          468
alone          468
dtype: int64468 people have an age greater than 23.
Final Answer: 468

> Finished chain.


{'input': 'How many people have more than 23 age', 'output': '468'}

## **06: Multi DataFrame Example**

### **Agent can also interact with Multiple DataFrames passed in a list**

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   who          891 non-null    object 
 10  adult_male   891 non-null    bool   
 11  deck         203 non-null    object 
 12  embark_town  889 non-null    object 
 13  alive        891 non-null    object 
 14  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB


In [19]:
df1=df.copy()

In [20]:
df1

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [28]:
df1["age"] = df1["age"].fillna(df1["age"].mean())

In [29]:
agent=create_pandas_dataframe_agent(llm, [df, df1], verbose=True,allow_dangerous_code=True)

In [30]:
agent.run("How many rows in the age column are different")



> Entering new AgentExecutor chain...
Thought: I need to compare the age column in both dataframes
Action: python_repl_ast
Action Input: df1['age'].equals(df2['age'])FalseI need to find the number of rows where the age column is different
Action: python_repl_ast
Action Input: len(df1[df1['age'] != df2['age']])177177 rows in the age column are different
Final Answer: 177

> Finished chain.


'177'

In [31]:
df2=df1.copy()

In [32]:
df2["Age_Multiplied"]=df1["age"] * 2

In [33]:
df2.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,Age_Multiplied
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,44.0
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,76.0
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,52.0
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,70.0
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,70.0


In [34]:
agent=create_pandas_dataframe_agent(llm, [df,df1, df2], verbose=True, allow_dangerous_code=True)

In [35]:
agent.invoke("Are the number of columns same in all the dataframe")



> Entering new AgentExecutor chain...
Thought: I need to check the number of columns in each dataframe
Action: python_repl_ast
Action Input: len(df1.columns)1515 columns in df1
Action: python_repl_ast
Action Input: len(df2.columns)1515 columns in df2
Action: python_repl_ast
Action Input: len(df3.columns)1616 columns in df3
Action: python_repl_ast
Action Input: len(df1.columns) == len(df2.columns) == len(df3.columns)False15 columns in df1 and df2, but 16 columns in df3
Final Answer: No, the number of columns is not the same in all the dataframes.

> Finished chain.


'No, the number of columns is not the same in all the dataframes.'